In [ ]:
# 기본적인 임포트
import matplotlib.pyplot as plt
import numpy as np

# 과대적합과 과소적합

과대적합이란 모델이 훈련세트에만 좋은 성능을 내고 검증세트에는 나쁜 성능을 내는 경우다.  
일반적으로, 훈련세트 정확도가 99%, 검증세트 정확도가 80%라면 과대적합을 의심할 수 있다.

</br>

과소적합은 그냥 성능자체가 구린경우다.  

</br>

즉, 우리는 성능도 우수하면서 검증세트 정확도와 훈련세트 정확도의 차이가 거의 없는  
모델을 만드는 것이 훌륭한 모델을 만드는 것이라 할 수 있다.

## 에포크 손실함수을 통한 과대적합과 과소적합 분석하기


In [ ]:
# 지난 시간에 만든 코드
class SingleLayer:
# 지난시간에 만든 SingleLayer 클래스 수정
  def __init__(self, learning_rate=0.1, l1=0, l2=0):
      self.w = None
      self.b = None
      self.losses = []
      self.val_losses = []
      self.w_history = []
      self.lr = learning_rate
      self.l1 = l1
      self.l2 = l2

  def forpass(self, x):
    z = np.sum(x * self.w) + self.b   
    return z

  def backprop(self, x, err):
    w_grad = x * err                 
    b_grad = 1 * err                 
    return w_grad, b_grad

  def fit(self, x, y, epochs=100):
    self.w = np.ones(x.shape[1])
    self.b = 0
    self.w_history.append(self.w.copy())
    np.random.seed(42)
    for i in range(epochs):
      loss = 0
      indexes = np.random.permutation(np.arange(len(x)))
      for i in indexes:
        z = self.forpass(x[i])
        a = self.activation(z)
        err = -(y[i] - a)
        w_grad, b_grad = self.backprop(x[i], err)
        self.w -= self.lr * w_grad
        self.b -= b_grad
        # 가중치를 기록합니다.
        self.w_history.append(self.w.copy())
        # 안전한 로그 계산을 위해 클리핑한 후 손실을 누적한다.  
        a = np.clip(a, 1e-10, 1-1e-10)
        loss += -(y[i] * np.log(a) + (1-y[i]) * np.log(1-a))
      # 에포크 마다 평균 손실을 저장합니다.
      self.losses.append(loss/len(y))

  # 시그모이드 함수
  def activation(self, z):
    z = np.clip(z, -100, None)
    a = 1 / (1 + np.exp(-z))
    return a

  def predict(self, x):
    z = [self.forpass(x_i) for x_i in x]    # 정방향 계산
    return np.array(z) > 0                  # 계단함수 적용

  def score(self, x, y):            # 정확도를 계산해주는 메서드
    return np.mean(self.predict(x) == y)

  def update_val_loss(self, x_val, y_val):
    if x_val is None:
      return
    val_loss = 0
    for i in range(len(x_val)):
      z = self.forpass(x_val[i])        # 정방향 계산
      a = self.activation(z)            # 활성화 함수 적용
      a = np.clip(a, 1e-10, 1-1e-10)    
      val_loss += -(y_val[i] * np.log(a) + (1 - y_val[i]) * np.log(1-a))
    self.val_losses.append(val_loss/len(y_val) + self.reg_loss())